In [1]:
# %pip install transformers datasets

In [2]:
import numpy as np
import torch
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# DATA
# dataset = load_dataset('emotion')
dataset = load_dataset('tamilmixsentiment')
dataset


# PARAMS
num_labels = len(set(dataset['train']['label']))
print('num_labels:', num_labels)

batch_size, model_ckpt = 64, 'distilbert-base-uncased'
batch_size, model_ckpt = 32, 'bert-base-multilingual-cased'
# batch_size, model_ckpt = 32, 'xlm-roberta-large'


# DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


# TOKENS
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)


dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)
dataset_encoded


print(dataset_encoded['train'].format)
dataset_encoded.set_format('torch')
print(dataset_encoded['train'].format)


# MODEL
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=num_labels
).to(device)

print('model.device', model.device)


def compute_metrics(pred):
    res = {'acc': 1.0, 'F1': 2.0}
    return res


training_args = TrainingArguments(
    output_dir='resultados',
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    disable_tqdm=False,
    evaluation_strategy='steps',
    logging_steps=batch_size,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded['train'],
    eval_dataset=dataset_encoded['validation'],
    tokenizer=tokenizer,
)

print('trainer.args.device:', trainer.args.device)

Found cached dataset tamilmixsentiment (/home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390)


  0%|          | 0/3 [00:00<?, ?it/s]

num_labels: 5


Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-cb7d863ba2d9264d.arrow
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-b98de94a7d12a2ad.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

{'type': None, 'format_kwargs': {}, 'columns': ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'output_all_columns': False}
{'type': 'torch', 'format_kwargs': {}, 'columns': ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'output_all_columns': False}


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

model.device cuda:0
trainer.args.device: cuda:0


In [4]:
trainer.train()
trainer.save_model('resultados2')

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11335
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 710
  Number of trainable parameters = 177857285
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` meth

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 14.76 GiB total capacity; 2.79 GiB already allocated; 16.75 MiB free; 2.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
preds = trainer.predict(dataset_encoded['validation'])

In [ ]:
preds

In [ ]:
preds.metrics

In [ ]:
preds.predictions

In [ ]:
np.argmax(preds.predictions, axis=1)

In [ ]:
pipe = pipeline("text-classification", 'resultados2')

In [ ]:
res = pipe(dataset['validation']['text'])

In [ ]:
res